source: https://www.freecodecamp.org/news/creating-your-first-image-classifier/

# 1. Setup

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

%config InlineBackend.figure_format = 'retina'

In [134]:
from fastai.vision import *
from fastai.metrics import error_rate
from pathlib import Path
from glob2 import glob
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from PIL import Image

import matplotlib.pyplot as plt
import zipfile as zf
import seaborn as sns
import pandas as pd
import numpy as np
import tensorflow.keras
import shutil
import sys
import csv
import os
import re

from sklearn.model_selection import train_test_split 
from tensorflow.keras.utils import to_categorical 

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D 
from tensorflow.keras.layers import Dense, Dropout 
from tensorflow.keras.layers import Flatten, BatchNormalization


# 2. Import Data

Extract contents of zipfile

In [3]:
files = zf.ZipFile("dataset-resized.zip",'r')
files.extractall()
files.close()

print(files)

<zipfile.ZipFile [closed]>


Helper functions to sort data into folders

In [4]:
## helper functions ##

## splits indices for a folder into train, validation, and test indices with random sampling
    ## input: folder path
    ## output: train, valid, and test indices    
def split_indices(folder,seed1,seed2):    
    n = len(os.listdir(folder))
    full_set = list(range(1,n+1))

    ## train indices
    random.seed(seed1)
    train = random.sample(list(range(1,n+1)),int(.5*n))

    ## temp
    remain = list(set(full_set)-set(train))

    ## separate remaining into validation and test
    random.seed(seed2)
    valid = random.sample(remain,int(.5*len(remain)))
    test = list(set(remain)-set(valid))
    
    return(train,valid,test)

## gets file names for a particular type of trash, given indices
    ## input: waste category and indices
    ## output: file names 
def get_names(waste_type,indices):
    file_names = [waste_type+str(i)+".jpg" for i in indices]
    return(file_names)    

## moves group of source files to another folder
    ## input: list of source files and destination folder
    ## no output
def move_files(source_files,destination_folder):
    for file in source_files:
        shutil.move(file,destination_folder)

Structure directory into train, test and split

In [5]:
## paths will be train/cardboard, train/glass, etc...
subsets = ['train','valid']
waste_types = ['cardboard','glass','metal','paper','plastic','trash']

## create destination folders for data subset and waste type
for subset in subsets:
    for waste_type in waste_types:
        folder = os.path.join('data',subset,waste_type)
        if not os.path.exists(folder):
            os.makedirs(folder)
            
if not os.path.exists(os.path.join('data','test')):
    os.makedirs(os.path.join('data','test'))
            
## move files to destination folders for each waste type
for waste_type in waste_types:
    source_folder = os.path.join('dataset-resized',waste_type)
    train_ind, valid_ind, test_ind = split_indices(source_folder,1,1)
    
    ## move source files to train
    train_names = get_names(waste_type,train_ind)
    train_source_files = [os.path.join(source_folder,name) for name in train_names]
    train_dest = "data/train/"+waste_type
    move_files(train_source_files,train_dest)
    
    ## move source files to valid
    valid_names = get_names(waste_type,valid_ind)
    valid_source_files = [os.path.join(source_folder,name) for name in valid_names]
    valid_dest = "data/valid/"+waste_type
    move_files(valid_source_files,valid_dest)
    
    ## move source files to test
    test_names = get_names(waste_type,test_ind)
    test_source_files = [os.path.join(source_folder,name) for name in test_names]
    ## I use data/test here because the images can be mixed up
    move_files(test_source_files,"data/test")

Get a path to the folder containing the images

In [131]:
# train_path = Path(os.getcwd())/"data/train"
# test_path = Path(os.getcwd())/"data/test"
# valid_path = Path(os.getcwd())/"data/valid"
path = Path(os.getcwd())/"dataset-resized"
print(path)

/Users/RebeccaHallam/OneDrive - Imperial College London/University/EE4 (19-20)/Final Year Project/App/Model/dataset-resized


# 3. Data Preprocessing

Generate list of files

In [132]:
def createFileList(myDir, format='.jpg'):
    fileList = []
    print(myDir)
    for root, dirs, files in os.walk(myDir, topdown=False):
        for name in files:
            if name.endswith(format):
                fullName = os.path.join(root, name)
                fileList.append(fullName)
    return fileList

fileList_train = createFileList(train_path,format='.jpg')
fileList_test = createFileList(test_path,format='.jpg')
fileList_valid = createFileList(valid_path,format='.jpg')
fileList = createFileList(path,format='.jpg')

/Users/RebeccaHallam/OneDrive - Imperial College London/University/EE4 (19-20)/Final Year Project/App/Model/data/train
/Users/RebeccaHallam/OneDrive - Imperial College London/University/EE4 (19-20)/Final Year Project/App/Model/data/test
/Users/RebeccaHallam/OneDrive - Imperial College London/University/EE4 (19-20)/Final Year Project/App/Model/data/valid
/Users/RebeccaHallam/OneDrive - Imperial College London/University/EE4 (19-20)/Final Year Project/App/Model/dataset-resized


In [133]:
print(len(fileList_train))
print(len(fileList_test))
print(len(fileList_valid))
print(len(fileList))

1262
635
630
0


dictionary of labels

In [172]:
label_dict = {
    1: "paper",
    2: "cardboard",
    3: "trash",
    4: "plastic",
    5: "metal",
    6: "glass",
    7: "unknown",
}


Convert to images to matrix

In [126]:
def stringContains(filePath):
    if "paper" in filePath:
        return 1
    elif "cardboard" in filePath:
        return 2
    elif "trash" in filePath:
        return 3
    elif "plastic" in filePath:
        return 4
    elif "metal" in filePath:
        return 5
    elif "glass" in filePath:
        return 6
    else:
        return 7

def convertToMatrix(fileList,num_images):
    
    num_pixels = 196608
    data = np.empty((num_images,num_pixels))
    labels = []
    index = 0
    
    for file in fileList:
        
        img = Image.open(file) # open image
        arr = np.array(img.convert('L')) # convert to grey and place in numpy array
        vec = arr.ravel() # flatten to vector
        data[index,:] = vec.T # add to matrix
        
        name = stringContains(img.filename) # get image name
        labels = np.append(labels,name) # add to array
        
        index += 1
    return data, labels


train,train_l = convertToMatrix(fileList_train,1262)
test,test_l = convertToMatrix(fileList_test,635)
valid,valid_l = convertToMatrix(fileList_valid,630)

train = train.astype('float32')
train_l = train_l.astype(int)
test = test.astype('float32')
test_l = test_l.astype(int)
valid = valid.astype('float32')
valid_l = valid_l.astype(int)

print(train.shape)
print(test.shape)
print(valid.shape)

(1262, 196608)
(635, 196608)
(630, 196608)


convert categories (labels) into one hot encoding

In [123]:
train_labels = to_categorical(train_l)
test_labels = to_categorical(test_l)
valid_labels = to_categorical(valid_l)

print(train_labels.shape)
print(test_labels.shape)
print(valid_labels.shape)

(1262, 7)
(635, 7)
(630, 7)


reshape data

In [164]:
rows = 384
cols = 512
input_shape = (rows,cols,1)

train_data = train.reshape(train.shape[0],rows,cols,1)
test_data = test.reshape(test.shape[0],rows,cols,1)
valid_data = valid.reshape(valid.shape[0],rows,cols,1)

normalise image data

In [165]:
train_data /= 255.0
test_data /= 255.0
valid_data /= 255.0

# 4. Train Model

In [169]:
model = Sequential()
# 32 layers, 5x5 convolutional layer window, 2x2 pooling layer window
model.add(Conv2D(32,(5,5),activation='relu',input_shape=input_shape))
model.add(MaxPooling2D(2,2))
model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 380, 508, 32)      832       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 190, 254, 32)      0         
Total params: 832
Trainable params: 832
Non-trainable params: 0
_________________________________________________________________


In [174]:
# num_classes = 7
# x = tensorflow.placeholder("float",[None,rows,cols,1])
# y = tensorflow.placeholder("float",[None,num_classes])

model = Sequential()
model.add(Conv2D(32,(5,5),activation='relu',input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 380, 508, 32)      832       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 190, 254, 32)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 186, 250, 64)      51264     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 93, 125, 64)       0         
Total params: 52,096
Trainable params: 52,096
Non-trainable params: 0
_________________________________________________________________


In [175]:
# add a densely connected layer
model.add(Dense(7,activation='softmax'))
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 380, 508, 32)      832       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 190, 254, 32)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 186, 250, 64)      51264     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 93, 125, 64)       0         
_________________________________________________________________
dense_8 (Dense)              (None, 93, 125, 7)        455       
Total params: 52,551
Trainable params: 52,551
Non-trainable params: 0
_________________________________________________________________


In [179]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
model.fit(train_data, train_labels,
          batch_size=1262,
          epochs=5)
test_loss, test_acc = model.evaluate(test_data, test_labels)
print("Test accuracy:", test_acc)

ValueError: A target array with shape (1262, 7) was passed for an output of shape (None, 93, 125, 7) while using as loss `categorical_crossentropy`. This loss expects targets to have the same shape as the output.